![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# Exploring RGB Visualzations of SAR and Deriving SAR Time Series Metrics

### Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, [Earth Big Data, LLC](http://earthbigdata.com/) 

<img style="padding:7px;" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /></font>

This notebook introduces you to some popular RGB visualizations of multi-temporal and multi-polarization SAR data. The exercise is done in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. 

**This notebook covers the following data analysis concepts:**

- How to create RGB visualizations of multi-temporal SAR data
- How to interprete these RGB images
- Popular RGB visualizations of dual-polarization SAR data
- How do derive time-series metrics from deep SAR data stacks
- How to export RGB products as GeoTiffs for visualization in a GIS


---

**<font face="Calibri" size="5" color='rgba(200,0,0,0.2)'>Important Note about JupyterHub</font>**

**Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.**

In [ ]:

%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
// alert(command)
kernel.execute(command)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))


---

## **0. Importing Relevant Python Packages**
In this notebook we will use the following scientific libraries:

1. **[Pandas](https://pandas.pydata.org/)** is a Python library that provides high-level data structures and a vast variety of tools for analysis. The great feature of this package is the ability to translate rather complex operations with data into one or two commands. Pandas contains many built-in methods for filtering and combining data, as well as the time-series functionality. 

1. **[GDAL](https://www.gdal.org/)** is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.

1. **[NumPy](http://www.numpy.org/)** is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects.

1. **[Matplotlib](https://matplotlib.org/index.html)** is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib.

1. The **[asf-hyp3 API](https://www.pydoc.io/pypi/asf-hyp3-1.1.1/index.html)** provides useful functions and scripts for accessing and processing SAR data via the Alaska Satellite Facility's Hybrid Pluggable Processing Pipeline, or HyP3 (pronounced "hype").

1. **[SciPY](https://www.scipy.org/about.html)** is a library that provides functions for numerical integration, interpolation, optimization, linear algebra and statistics.

Our first step is to **import them:**

In [ ]:
%%capture
from pathlib import Path

import pandas as pd # for DatetimeIndex
from osgeo import gdal

from IPython.display import HTML

%matplotlib inline
import matplotlib.pylab as plb # for figure, grid, rcParams, savefig
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import rc
import numpy as np
from skimage import exposure # to enhance image display

import asf_notebook as asfn
asfn.jupytertheme_matplotlib_format()


## **1. Load Data Stack <img src="NotebookAddons/Deforest-MadreDeDios.jpg" width="350" style="padding:7px;" align="right" />**

This notebook will be using a 78-image deep dual-polarization C-band SAR data stack over Madre de Dios in Peru to demonstrate how to create color composites from multi temporal and dual-polarization SAR data and how to derive higher level parameters such as the multi-temporal mean, the coefficient of variation, and others. The C-band data were acquired by ESA's Sentinel-1 SAR sensor constellation and are available to you through the services of the [Alaska Satellite Facility](https://www.asf.alaska.edu/).

The site in question is interesting as it has experienced extensive logging over the last 10 years (see image to the right; [Monitoring of the Andean Amazon Project](https://blog.globalforestwatch.org/)). Since the 1980s, people have been clearing forests in this area for farming, cattle ranching, logging, and (recently) gold mining. Creating RGB color composites is an easy way to visualize ongoing changes in the landscape.

Before we get started, let's first **create a working directory for this analysis and change into it:** 

In [ ]:
path = Path("/home/jovyan/notebooks/SAR_Training/English/Ecosystems/S1-MadreDeDios")

if not path.exists():
    path.mkdir()

We will **retrieve the relevant data** from an [Amazon Web Service (AWS)](https://aws.amazon.com/) cloud storage bucket **using the following command**:

In [ ]:
time_series_path = 's3://asf-jupyter-data/MadreDeDios.zip'
time_series = Path(time_series_path).name
!aws --region=us-east-1 --no-sign-request s3 cp $time_series_path $time_series

Now, let's **unzip the file (overwriting previous extractions) and clean up after ourselves:**

In [ ]:
if Path(time_series).exists():
    asfn.asf_unzip(str(path), time_series)
    Path(time_series).unlink()   

## **2. Define Data Directory and Path to VRT**
**Create a variable containing the VRT filename and the image acquisition dates:**

In [ ]:
!gdalbuildvrt -separate {path}/raster_stack.vrt {path}/tiffs/*_VV.tiff
image_file_VV = f"{path}/raster_stack.vrt"

**Create an index of timedelta64 data with Pandas:**

In [ ]:
!ls {path}/tiffs/*_VV.tiff | sort | sed 's/[^0-9]*//g' | cut -c 2-9  > {path}/raster_stack_VV.dates

datefile_VV = f'{path}/raster_stack_VV.dates'
dates_VV = open(datefile_VV).readlines()
tindex_VV = pd.DatetimeIndex(dates_VV)

**Print the bands and dates for all images in the virtual raster table (VRT):**

In [ ]:
print(f"Bands and dates for {image_file_VV}")
for i, d in enumerate(tindex_VV):
    print("{:4d} {}".format(i+1, d.date()), end=' ')
    if (i+1)%5 == 1:
        print()


---

## **3. Open Your Data Stack and Visualize Some Layers**

We will **open your VRT** and visualize some layers using Matplotlib. 

In [ ]:
img = gdal.Open(image_file_VV)

**Print the bands, pixels, and lines:**

In [ ]:
print(f"Number of  bands: {img.RasterCount}")
print(f"Number of pixels: {img.RasterXSize}")
print(f"Number of  lines: {img.RasterYSize}")

**Read in raster data for the first two bands:**

In [ ]:
raster_1 = img.GetRasterBand(1).ReadAsArray() # change the number passed to GetRasterBand() to 
where_are_NaNs = np.isnan(raster_1)           # read rasters from different bands
raster_1[where_are_NaNs] = 0

raster_2 = img.GetRasterBand(78).ReadAsArray() #must pass a valid band number to GetRasterBand()
where_are_NaNs = np.isnan(raster_2)
raster_2[where_are_NaNs] = 0

**Plot images and histograms for bands 1 and 2:**<br>
Note: Depending the histograms plotted by this cell, you may wish to adjust `vmax` when calling `imshow()` on `ax1` and `ax3`. Increase the `vmax` value if the histogram cuts off much of the end of the peak, making your image too bright to see features well. Decrease `vmax` if the histogram extends much beyond the end of the peak, which will make your image appear dark.

In [ ]:
# Setup the pyplot plots
plt.rcParams.update({'font.size': 14})
fig = plb.figure(figsize=(18, 10)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 221 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 222 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot
fig.subplots_adjust(hspace=.35)


# Plot the band 1 image
band_number = 1
ax1.imshow(raster_1, cmap='gray', vmin=0, vmax=0.3) #,vmin=2000,vmax=10000)
ax1.set_title(f'Image Band {band_number} {tindex_VV[band_number-1].date()}')

# Flatten the band 1 image into a 1 dimensional vector and plot the histogram:
h = ax2.hist(raster_1.flatten(), bins=200, range=(0, 0.3))
ax2.xaxis.set_label_text('Calibrated Radar Cross Section (RCS) [Power Scale]')
ax2.set_title(f'Histogram Band {band_number} {tindex_VV[band_number-1].date()}')

# Plot the band 2 image
band_number = 78
ax3.imshow(raster_2, cmap='gray', vmin=0, vmax=0.3) #,vmin=2000,vmax=10000)
ax3.set_title(f'Image Band {band_number} {tindex_VV[band_number-1].date()}')

# Flatten the band 2 image into a 1 dimensional vector and plot the histogram:
h = ax4.hist(raster_2.flatten(), bins=200, range=(0, 0.3))
ax4.xaxis.set_label_text('Calibrated Radar Cross Section (RCS) [Power Scale]')
ax4.set_title(f'Histogram Band {band_number} {tindex_VV[band_number-1].date()}')



---

## **4. Create a Time Series Animation to get an Idea of the Dynamics at the Site**

### 4.1 Load Time Series Stack

Now we are ready to create a time series animation from the calibrated SAR data.

**First, create a raster from band 0 and a raster stack from all the images:**

In [ ]:
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack_VV = img.ReadAsArray()

### 4.2 Calibration and Data Conversion between dB and Power Scales 

**<font color='rgba(200,0,0,0.2)'>Note, that if your data were generated by HyP3, this step is not necessary! HyP3 performs the full data calibration and provides you with calibrated data in power scale.</font>**
    
If, your data is from a different source, however, calibration may be necessary to ensure that image gray values correspond to proper radar cross section information. 

Calibration coefficients for SAR data are often defined in the decibel (dB) scale due to the high dynamic range of the imaging system. For the L-band ALOS PALSAR data at hand, the conversion from uncalibrated DN values to calibrated radar cross section values in dB scale is performed by applying a standard **calibration factor of -83 dB**. 

$\gamma^0_{dB} = 20 \cdot log10(DN) -83$

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric.

**To apply the calibration constant for your data and export in *dB* scale, uncomment the following code cell**:

In [ ]:
 #caldB=20*np.log10(rasterstack)-83

While **dB**-scaled images are often "visually pleasing", they are often not a good basis for mathematical operations on data. For instance, when we compute the mean of observations, it makes a difference whether we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. 
    
Please note that the **correct scale** in which operations need to be performed **is the power scale.** This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed.

To **convert from dB to power**, apply: $\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$ 

In [ ]:
#calPwr=np.power(10.,caldB/10.)


### 4.3 Create Time Series Animation 

**Create and move into a directory in which to store our plots and animations:**

In [ ]:
product_path = path/'plots_and_animations'

if not product_path.exists():
    product_path.mkdir()

In [ ]:
%%capture 
fig = plt.figure(figsize=(8, 8))
ax = fig.subplots()
ax.axis('off')
vmin = np.percentile(rasterstack_VV.flatten(), 5)
vmax = np.percentile(rasterstack_VV.flatten(), 95)

r0dB = 20 * np.log10(raster0) - 83

im = ax.imshow(raster0, cmap='gray', vmin=vmin, vmax=vmax)
ax.set_title("{}".format(tindex_VV[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex_VV[i].date()))
    im.set_data(rasterstack_VV[i])

# Interval is given in milliseconds
ani = animation.FuncAnimation(fig, animate, frames=rasterstack_VV.shape[0], interval=400)

**Configure matplotlib's RC settings for the animation:**

In [ ]:
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

**Create a javascript animation of the time-series running inline in the notebook:**

In [ ]:
HTML(ani.to_jshtml())

**Delete the dummy png** that was saved to the current working directory while generating the javascript animation in the last code cell.

In [ ]:
try:
    tmp_rm = product_path/'None0000000.png'
    tmp_rm.unlink()
except FileNotFoundError:
    pass

**Save the animation (animation.gif):**

In [ ]:
ani.save(product_path/'animation.gif', writer='pillow', fps=2)


---

## **5. Create RGB Visualization from Multi-Temporal SAR Images**

Now we are ready to create our first RGB visualization. We will pick images from different dates/times/years to form an RGB. The colors in the RGB will then provide information on changes that occurred during the time span covered by the data.

**First, we mask out pixels without relevant information to be unbiased in statical number we calculate later:**

In [ ]:
mask = rasterstack_VV == 0
rasterPwr = np.ma.array(rasterstack_VV, mask=mask, dtype=np.float32)

We make an RGB stack to display the first, center, and last time step of our available stack as a multi-temporal color composite. The `np.dstack` results in an array of the form `[lines,pixels,bands]`, which is the format we need for RGB display with matplotlib's `imshow()` function. Note that numpy array indexing starts with 0, so band 1 is `raster[0]`.

In [ ]:
rgb_bands = (1, int(img.RasterCount/2), img.RasterCount)  # first, center, last band

rgb_idx = np.array(rgb_bands)-1  # get array index from bands by subtracting 1

rgb = np.dstack((rasterPwr[rgb_idx[0]], 
                 rasterPwr[rgb_idx[1]], 
                 rasterPwr[rgb_idx[2]]))

rgb_dates = (tindex_VV[rgb_idx[0]].date(),
             tindex_VV[rgb_idx[1]].date(), 
             tindex_VV[rgb_idx[2]].date())

We are also interested in displaying the image enhanced with histogram equalization. We can use the function `exposure.equalize_hist()` from the `skimage.exposure` module.

In [ ]:
rgb_stretched = rgb.copy()
# For each band we apply the strech
for i in range(rgb_stretched.shape[2]):
    rgb_stretched[:,:,i] = exposure.\
    equalize_hist(rgb_stretched[:,:,i].data,
    mask =~ np.equal(rgb_stretched[:,:,i].data,0.))

Now let's **display the the unstrechted and histogram equalized images side by side.**

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('Multi-temporal Sentinel-1 backscatter image R:{} G:{} B:{}'
             .format(rgb_dates[0],rgb_dates[1],rgb_dates[2]))
plt.axis('off')
ax[0].imshow(rgb)
ax[0].set_title('Unstreched')
ax[0].axis('off')
ax[1].imshow(rgb_stretched)
ax[1].set_title('Histogram Equalized')
_ = ax[1].axis('off')

<div class="alert alert-success">

<span style='font-size:20pt; font-weight:bold; color:red'><u>EXERCISE</u>: </span> Pick Different Time Steps and Interpret Resulting RGB Visualization

To get a feeling for the RGB visualizations, please explore the 78 image deep data stack a bit more. Pick different mutli-temporal image layers and form more RGB visualizations. Interpret the colors you see in terms of the change that likely occurred on the ground.
</div>

<br>

---

Ideally, you may want to analyze these images in a GIS framework, where you can combine the SAR RGBs with other relevant data layers. To do so, we will export **your favorite multi-temporal RGB image** as a GeoTiff. First, we **write a function to convert our plots into GeoTiffs:**

In [ ]:
vrt = gdal.Open(image_file_VV)
vrt_info = gdal.Info(vrt, format='json')
coords = [vrt_info['cornerCoordinates']['upperLeft'], 
          vrt_info['cornerCoordinates']['lowerRight']]
print(coords)

In [ ]:
utm_zone = vrt_info['coordinateSystem']['wkt'].split('ID')[-1].split(',')[1][:-2]
print(f"utm_zone: {utm_zone}")

In [ ]:
# do not include a file extension in out_filename
# extent must be in the form of a list: [[upper_left_x, upper_left_y], [lower_right_x, lower_right_y]]
def geotiff_from_plot(source_image, out_filename, extent, utm_zone, cmap=None, vmin=None, vmax=None, interpolation=None, dpi=300):
    plt.figure()
    plt.axis('off')
    plt.imshow(source_image, cmap=cmap, vmin=vmin, vmax=vmax, interpolation=interpolation)
    temp = f"{out_filename}_temp.png"
    plt.savefig(temp, dpi=dpi, transparent='true', bbox_inches='tight', pad_inches=0)

    cmd = f"gdal_translate -of Gtiff -a_ullr {extent[0][0]} {extent[0][1]} {extent[1][0]} {extent[1][1]} -a_srs EPSG:{utm_zone} {temp} {out_filename}.tiff"
    !{cmd}
    try:
        Path(temp).unlink()
    except FileNotFoundError:
        pass

**Now we can save your last RGB image as a GeoTIFF (`MadreDeDios-RGB.tiff`):**

In [ ]:
%%capture
geotiff_from_plot(rgb_stretched, product_path/'MadreDeDios-multitemp-RGB', coords, utm_zone)


---

## **6. Create RGB Visualization from Dual-Pol SAR Images**

Now let's demonstrate another popular RGB visualization, one that is particularly useful for dual-pol SAR data such as Sentinel-1. The color composites are constructed with co-polarized (VV-channel) data assigned to the red channel, cross-polarized (VH) data to the green channel, and the co-/cross-polarized ratio (VV/VH ratio) to the blue channel. A nice effect for forest applications with this color assignment strategy is that forests tend to be shown in shades of green, and typically the brightness of green corresponds to the amount of biomass in the forest. Also, water tends to be represented in blue colors, which also represents other surface scattering components. Naturally, different histogram stretches may be applied to enhance various surface components.
<br><br>

**To create this visualization, we first need to load the cross-polarized VH data into the notebook:**

In [ ]:
!gdalbuildvrt -separate {path}/raster_stack_VH.vrt {path}/tiffs/*_VH.tiff
image_file_VH = path/"raster_stack_VH.vrt"

In [ ]:
!ls {path}/tiffs/*_VH.tiff | sort | sed 's/[^0-9]*//g' | cut -c 2-9 > {path}/raster_stack_VH.dates
datefile_VH = path/'raster_stack_VH.dates'
dates_VH = open(str(datefile_VH)).readlines()
tindex_VH = pd.DatetimeIndex(dates_VH)

In [ ]:
if image_file_VH.exists():
    print(f"Bands and dates for {image_file_VH}")
    for i, d in enumerate(tindex_VH):
        print("{:4d} {}".format(i+1, d.date()),end=' ')
        if (i+1)%5 == 1: print()

**Now we load the VH data stack and prepare it for visualization:**

In [ ]:
img_VH = gdal.Open(str(image_file_VH))
band_VH = img_VH.GetRasterBand(1)
raster0_VH = band_VH.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack_VH = img_VH.ReadAsArray()
mask = rasterstack_VH == 0
rasterPwr_VH = np.ma.array(rasterstack_VH, mask=mask, dtype=np.float32)

**<font color='rgba(200,0,0,0.2)'>Pick a band number</font> you would like to visualize:**

In [ ]:
#Pick a band number for visualization
bandno = 1

**Now we can create an RGB image using the mentioned color assignments (R: VV; G: VH; B: VV/VH), color stretch the RGB, and visualize it:**

In [ ]:
rgb_band = (bandno, bandno, bandno)  # first, center, last band
rgb_idx = np.array(rgb_bands)-1  # get array index from bands by subtracting 1
rgb = np.dstack((rasterPwr[rgb_idx[0]], rasterPwr_VH[rgb_idx[0]], rasterPwr[rgb_idx[0]]/rasterPwr_VH[rgb_idx[0]]))
rgb_date = (tindex_VH[rgb_idx[0]].date())

In [ ]:
rgb_stretched_POL = rgb.copy()
# For each band we apply the strech
for i in range(rgb_stretched_POL.shape[2]):
    rgb_stretched_POL[:,:,i] = exposure.\
    equalize_hist(rgb_stretched_POL[:,:,i].data,
    mask =~ np.equal(rgb_stretched_POL[:,:,i].data,0.))

In [ ]:
fig = plb.figure(figsize=(16, 16)) # Initialize figure with a size
ax1 = fig.add_subplot(221)  # 221 determines: 2 rows, 2 plots, first plot
ax2 = fig.add_subplot(222)  # 222 determines: 2 rows, 2 plots, second plot
ax3 = fig.add_subplot(223)  # 223 determines: 2 rows, 2 plots, third plot
ax4 = fig.add_subplot(224)  # 224 determines: 2 rows, 2 plots, fourth plot
plt.rcParams.update({'font.size': 14})

# Plot the VV band
r_1 = img.GetRasterBand(bandno).ReadAsArray()
vmin = np.percentile(r_1.flatten(), 5)
vmax = np.percentile(r_1.flatten(), 95)
ax1.imshow(r_1, cmap='gray', vmin=vmin, vmax=vmax) 
ax1.set_title('VV Channel')
ax1.axis('off')

# Plot the VH band
r_1 = img_VH.GetRasterBand(bandno).ReadAsArray()
vmin = np.percentile(r_1.flatten(), 5)
vmax = np.percentile(r_1.flatten(), 95)
ax2.imshow(r_1, cmap='gray', vmin=vmin, vmax=vmax)
ax2.set_title('VH Channel')
ax2.axis('off')

# Plot the VV/VH band
r_1 = img.GetRasterBand(bandno).ReadAsArray()/img_VH.GetRasterBand(bandno).ReadAsArray()
vmin = np.percentile(r_1.flatten(), 5)
vmax = np.percentile(r_1.flatten(), 95)
ax3.imshow(r_1, cmap='gray', vmin=vmin, vmax=vmax)
ax3.set_title('VV/VH Ratio Channel')
ax3.axis('off')

# Plot the RGB Composite band
ax4.imshow(rgb_stretched_POL)
ax4.set_title('Color Composite')
_ = ax4.axis('off')

We will again **export the RGB composite as a GeoTiff to allow further analysis in your favorite GIS environment:**

In [ ]:
%%capture
geotiff_from_plot(rgb_stretched_POL, product_path/'MadreDeDios-multipol-RGB', coords, utm_zone)


<br>
<div class="alert alert-success">

<!-- ### <font color='rgba(200,0,0,0.2)'> **<u>EXERCISE</u>**:</font> Pick Different Time Steps and Interpret Resulting Dual-Pol RGB Images  -->
### <span style='color:rgba(200,0,0,0.2)'> EXERCISE:</span> Pick Different Time Steps and Interpret Resulting Dual-Pol RGB Images 

Explore the 78 image deep data stack a bit more. Pick different time steps in the data stack and create additional RGB composites. You may see interesting changes in the color patterns. Answer the following questions for yourself:

- What are the green areas and what are different shades of green mean?
- Why is there fewer green in the visualization than you might have expected?
- What are the blue-shaded regions?
- What kind of differences in color patterns do you see between different time steps? What might be the reasons for these changes?
</div>
<br>


---

## **7. Plot the Time Series of Means Calculated Across the Subset**

To create the time series of means, we will go through the following steps:
1. Ensure that you use the data in **power scale** ($\gamma^o_{pwr}$) for your mean calculations.
1. compute means.
1. convert the resulting mean values into dB scale for visualization.
1. plot time series of means. 
<br><br>
**Compute the means:**

In [ ]:
rs_means_pwr_VH = np.mean(rasterPwr_VH, axis=(1, 2))
rs_means_pwr_VV = np.mean(rasterPwr, axis=(1, 2))

**Convert resulting mean value time-series to dB scale for visualization:**

In [ ]:
rs_means_dB_VH = 10.*np.log10(rs_means_pwr_VH)
rs_means_dB_VV = 10.*np.log10(rs_means_pwr_VV)

**Plot and save the time series of means (`RCSoverTime.png`):**

In [ ]:
# 3. Now let's plot the time series of means
plt.rcParams.update({'font.size': 14})
fig = plt.figure(figsize=(16, 4))
ax1 = fig.add_subplot(111)
ax1.plot(tindex_VV, rs_means_dB_VV)
ax1.set_xlabel('Date')
ax1.set_ylabel('VV Channel $\overline{\gamma^o}$ [power]')


ax2 = ax1.twinx()
ax2.plot(tindex_VH, rs_means_dB_VH, color='red')
ax2.set_ylabel('VH Channel $\overline{\gamma^o}$ [dB]')
fig.legend(['VV Channel', 'VH Channel'], loc=3)
plt.title('Time series profile of average band backscatter $\gamma^o$ ')
plt.savefig(product_path/"time_series_means", dpi=72, transparent='true')


---

## **8. Computation and Visualization of Time Series Metrics**

Once a time-series was constructed, we can compute **a set of metrics** that will aid us later in applications such as *change detection and active agriculture detection*. In the next code cells, we will compute the following variables for each pixel in the stack:

- Mean 
- Median
- Maximum
- Minimum
- Range (Maximum - Minimum)
- 5th Percentile
- 95th Percentile
- PRange (95th - 5th Percentile)
- Variance
- Coefficient of Variation (Variance/Mean)

---

Let's first **define a function for calculating the intended time-series metrics**:

In [ ]:
def timeseries_metrics(raster, ndv=0): 
    # Make us of numpy nan functions
    # Check if type is a float array
    if not raster.dtype.name.find('float') > -1:
        raster = raster.astype(np.float32)
    # Set ndv to nan
    if ndv != np.nan:
        raster[np.equal(raster,ndv)] = np.nan
    # Build dictionary of the metrics
    tsmetrics = {}
    rperc = np.nanpercentile(raster, [5, 50, 95], axis=0)
    tsmetrics['mean'] = np.nanmean(raster, axis=0)
    tsmetrics['max'] = np.nanmax(raster, axis=0)
    tsmetrics['min'] = np.nanmin(raster, axis=0)
    tsmetrics['range'] = tsmetrics['max'] - tsmetrics['min']
    tsmetrics['median'] = rperc[1]
    tsmetrics['p5'] = rperc[0]
    tsmetrics['p95'] = rperc[2]
    tsmetrics['prange'] = rperc[2] - rperc[0]
    tsmetrics['var'] = np.nanvar(raster, axis=0)
    tsmetrics['cov'] = tsmetrics['var'] / tsmetrics['mean']
    return tsmetrics

In [ ]:
metrics = timeseries_metrics(rasterPwr.filled(np.nan), ndv=np.nan)

In [ ]:
metric_keys = list(metrics.keys())
print(metric_keys)

Let's look at the histograms for the time series variance and coeficient of variation to aid displaying those images:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 4))
ax[0].hist(metrics['var'].flatten(), bins=100, range=(0, 0.001))
ax[1].hist(metrics['cov'].flatten(), bins=100, range=(0, 0.01))
_ = ax[0].set_title('Variance')
_ = ax[1].set_title('Coefficient of Variation')

We use thresholds determined from those histograms to set the scaling in the time series visualization. For the backscatter metrics we choose a typical range appropriate for this ecosystem and radar sensor. A typical range is -30 dB (0.0001) to -5.2 dB (0.3).

In [ ]:
# List the metrics keys you want to plot
fig = plt.figure(figsize=(16, 40))
for i, key in enumerate(metric_keys):
    ax = fig.add_subplot(5, 2, i+1)
    if key == 'var': 
        vmin, vmax = (0.0, 0.001)
    elif key == 'cov': 
        vmin, vmax = (0.0, 0.004)
    else:
        vmin, vmax = (0.0001, 0.3)
    ax.imshow(metrics[key], vmin=vmin, vmax=vmax, cmap='gray')
    ax.set_title(key.upper())
    ax.axis('off')

We will again **export the RGB composite as a GeoTiff to allow further analysis in your favorite GIS environment:**

In [ ]:
%%capture
Names = [] # List to keep track of all the names
for i in metric_keys:
    # Name, Array, DataType, NDV,bandnames=None,ref_image
    Name = product_path/f'MadreDeDios-{i}'
    print(Name)
    if i == 'var': 
        vmin, vmax = (0.0, 0.001)
    elif i == 'cov': 
        vmin, vmax = (0.0, 0.004)
    else:
        vmin, vmax = (0.0001, 0.3)
    geotiff_from_plot(metrics[i], Name, coords, utm_zone, cmap='gray', vmin=vmin, vmax=vmax)


---

## **8. Conclusion**

RGB Visualizations can be a powerful tool to visually analyze multi-temporal or multi-polarization SAR images. For multi-temporal data, the different color bands of an RGB composite relate to different time steps; hence, color tones represent changes of the landscape over time. 
    
For multi-polarization data, the RGB representation gives some indication of the type of surface cover present in an area. Areas shown in green in these composites usually are areas of high biomass while areas in blue are often regions covered in water. 

Please use caution when interpreting multi-polarization composites made from Sentinel-1 dual-pol data. As Sentinel-1 does not penetrate deep into vegetation, the composites often are not a correct representation of biomass in an area (composites don't look as green as they should).  

For a bit more information on change detection and SAR in general, please look at the recently published [SAR Handbook: Comprehensive Methodologies for Forest Monitoring and Biomass Estimation](https://gis1.servirglobal.net/TrainingMaterials/SAR/SARHB_FullRes.pdf).

---

<i>Exercise2-RGBandMetricsVisualization.ipynb - Version 1.4.1 - November 2021

**Version Changes:**

- Replaced `os` and obsolete `asfn` methods with `pathlib` counterparts
- Changed some bash script commands to properly attain dates
- Converted `html` to `Markdown`
</i>